In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

customer_df=pd.read_csv('we_fn_use_c_marketing_customer_value_analysis.csv')

In [2]:
##First look at its main features (head, shape, info).
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Customer                       9134 non-null   object 
 1   State                          9134 non-null   object 
 2   Customer Lifetime Value        9134 non-null   float64
 3   Response                       9134 non-null   object 
 4   Coverage                       9134 non-null   object 
 5   Education                      9134 non-null   object 
 6   Effective To Date              9134 non-null   object 
 7   EmploymentStatus               9134 non-null   object 
 8   Gender                         9134 non-null   object 
 9   Income                         9134 non-null   int64  
 10  Location Code                  9134 non-null   object 
 11  Marital Status                 9134 non-null   object 
 12  Monthly Premium Auto           9134 non-null   i

In [3]:
customer_df.head

<bound method NDFrame.head of      Customer       State  Customer Lifetime Value Response  Coverage  \
0     BU79786  Washington              2763.519279       No     Basic   
1     QZ44356     Arizona              6979.535903       No  Extended   
2     AI49188      Nevada             12887.431650       No   Premium   
3     WW63253  California              7645.861827       No     Basic   
4     HB64268  Washington              2813.692575       No     Basic   
...       ...         ...                      ...      ...       ...   
9129  LA72316  California             23405.987980       No     Basic   
9130  PK87824  California              3096.511217      Yes  Extended   
9131  TD14365  California              8163.890428       No  Extended   
9132  UP19263  California              7524.442436       No  Extended   
9133  Y167826  California              2611.836866       No  Extended   

     Education Effective To Date EmploymentStatus Gender  Income  ...  \
0     Bachelor      

In [5]:
customer_df.shape

(9134, 24)

In [6]:
##Rename the columns so they follow the PE8 (snake case).
customer_df.columns = customer_df.columns.str.lower()
customer_df.columns = customer_df.columns.str.replace(' ', '_')
customer_df.columns

Index(['customer', 'state', 'customer_lifetime_value', 'response', 'coverage',
       'education', 'effective_to_date', 'employmentstatus', 'gender',
       'income', 'location_code', 'marital_status', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies', 'policy_type',
       'policy', 'renew_offer_type', 'sales_channel', 'total_claim_amount',
       'vehicle_class', 'vehicle_size'],
      dtype='object')

In [7]:
##Change effective to date column to datetime format.
customer_df['effective_to_date']=pd.to_datetime(customer_df['effective_to_date'])
customer_df.effective_to_date

0      2011-02-24
1      2011-01-31
2      2011-02-19
3      2011-01-20
4      2011-02-03
          ...    
9129   2011-02-10
9130   2011-02-12
9131   2011-02-06
9132   2011-02-03
9133   2011-02-14
Name: effective_to_date, Length: 9134, dtype: datetime64[ns]

In [8]:
##Check NaN values per column
customer_df.isna().sum()

customer                         0
state                            0
customer_lifetime_value          0
response                         0
coverage                         0
education                        0
effective_to_date                0
employmentstatus                 0
gender                           0
income                           0
location_code                    0
marital_status                   0
monthly_premium_auto             0
months_since_last_claim          0
months_since_policy_inception    0
number_of_open_complaints        0
number_of_policies               0
policy_type                      0
policy                           0
renew_offer_type                 0
sales_channel                    0
total_claim_amount               0
vehicle_class                    0
vehicle_size                     0
dtype: int64

In [9]:
customer_df['customer'].duplicated().sum()

0

In [18]:
customer_df.select_dtypes(exclude=['object']).columns

Index(['customer_lifetime_value', 'effective_to_date', 'income',
       'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'total_claim_amount'],
      dtype='object')

In [19]:
##Define a function that differentiates between continuous and discrete variables. 
##Hint: A number of unique values might be useful. 
##Store continuous data into a continuous variable and do the same for discrete and categorical.
for a in customer_df.select_dtypes(exclude=['object']).columns: 
    if len(pd.value_counts(customer_df[a]))>len(customer_df)*0.2:
        print(a, ": continous")
    else: print(a, ": discrete")

customer_lifetime_value : continous
effective_to_date : discrete
income : continous
monthly_premium_auto : discrete
months_since_last_claim : discrete
months_since_policy_inception : discrete
number_of_open_complaints : discrete
number_of_policies : discrete
total_claim_amount : continous


In [35]:
##For the categorical data, check if there is some kind of text in a variable so we would need to clean it. 
##Hint: Use the same method you used in step 7. 
##Depending on the implementation, decide what to do with the variables you get.
for a in customer_df.select_dtypes(include=['object']).columns: 
    print(customer_df[a].value_counts())

BU79786    1
PU81096    1
CO75086    1
WW52683    1
XO38850    1
          ..
HS14476    1
YL91587    1
CT18212    1
EW35231    1
Y167826    1
Name: customer, Length: 9134, dtype: int64
California    3150
Oregon        2601
Arizona       1703
Nevada         882
Washington     798
Name: state, dtype: int64
No     7826
Yes    1308
Name: response, dtype: int64
Basic       5568
Extended    2742
Premium      824
Name: coverage, dtype: int64
Bachelor                2748
College                 2681
High School or Below    2622
Master                   741
Doctor                   342
Name: education, dtype: int64
Employed         5698
Unemployed       2317
Medical Leave     432
Disabled          405
Retired           282
Name: employmentstatus, dtype: int64
F    4658
M    4476
Name: gender, dtype: int64
Suburban    5779
Rural       1773
Urban       1582
Name: location_code, dtype: int64
Married     5298
Single      2467
Divorced    1369
Name: marital_status, dtype: int64
Personal Auto     67

In [37]:
##with customer_id column, we will need to drop it. 
customer_df.drop('customer', axis = 1)

state  customer_lifetime_value response  coverage education  \
0     Washington              2763.519279       No     Basic  Bachelor   
1        Arizona              6979.535903       No  Extended  Bachelor   
2         Nevada             12887.431650       No   Premium  Bachelor   
3     California              7645.861827       No     Basic  Bachelor   
4     Washington              2813.692575       No     Basic  Bachelor   
...          ...                      ...      ...       ...       ...   
9129  California             23405.987980       No     Basic  Bachelor   
9130  California              3096.511217      Yes  Extended   College   
9131  California              8163.890428       No  Extended  Bachelor   
9132  California              7524.442436       No  Extended   College   
9133  California              2611.836866       No  Extended   College   

     effective_to_date employmentstatus gender  income location_code  ...  \
0           2011-02-24         Employed      F   56274      Suburban  ...   
1           2011-01-31       Unemployed      F       0      Suburban  ...   
2           2011-02-19         Employed      F   48767      Suburban  ...   
3           2011-01-20       Unemployed      M       0      Suburban  ...   
4           2011-02-03         Employed      M   43836         Rural  ...   
...                ...              ...    ...     ...           ...  ...   
9129        2011-02-10         Employed      M   71941         Urban  ...   
9130        2011-02-12         Employed      F   21604      Suburban  ...   
9131        2011-02-06       Unemployed      M       0      Suburban  ...   
9132        2011-02-03         Employed      M   21941      Suburban  ...   
9133        2011-02-14       Unemployed      M       0      Suburban  ...   

     months_since_policy_inception  number_of_open_complaints  \
0                                5                          0   
1                               42                          0   
2                               38                          0   
3                               65                          0   
4                               44                          0   
...                            ...                        ...   
9129                            89                          0   
9130                            28                          0   
9131                            37                          3   
9132                             3                          0   
9133                            90                          0   

      number_of_policies     policy_type        policy  renew_offer_type  \
0                      1  Corporate Auto  Corporate L3            Offer1   
1                      8   Personal Auto   Personal L3            Offer3   
2                      2   Personal Auto   Personal L3            Offer1   
3                      7  Corporate Auto  Corporate L2            Offer1   
4                      1   Personal Auto   Personal L1            Offer1   
...                  ...             ...           ...               ...   
9129                   2   Personal Auto   Personal L1            Offer2   
9130                   1  Corporate Auto  Corporate L3            Offer1   
9131                   2  Corporate Auto  Corporate L2            Offer1   
9132                   3   Personal Auto   Personal L2            Offer3   
9133                   1  Corporate Auto  Corporate L3            Offer4   

     sales_channel total_claim_amount  vehicle_class vehicle_size  
0            Agent         384.811147   Two-Door Car      Medsize  
1            Agent        1131.464935  Four-Door Car      Medsize  
2            Agent         566.472247   Two-Door Car      Medsize  
3      Call Center         529.881344            SUV      Medsize  
4            Agent         138.130879  Four-Door Car      Medsize  
...            ...                ...            ...          ...  
9129           Web         